In [28]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

#beautifulsoup
import requests as req
from bs4 import BeautifulSoup as bs
from pprint import pprint

import re

In [29]:
driver = webdriver.Chrome(
)
driver.get("https://judgment.judicial.gov.tw/FJUD/Default_AD.aspx")

sleep(1)
input_case_reason = driver.find_element(By.CSS_SELECTOR, 'input#jud_jmain')
input_case_reason.send_keys("殺人")
input_case_content = driver.find_element(By.CSS_SELECTOR, 'input#jud_kw')
input_case_content.send_keys("中華民國刑法第 271 條&刑事判決")

sleep(2)

# 送出搜尋
search_button = driver.find_element(By.CSS_SELECTOR, 'input#btnQry')
search_button.click()
sleep(3)

In [30]:
#切到iframe中，抓page網址
iframe = driver.find_element(By.CSS_SELECTOR, "iframe#iframe-data")
driver.switch_to.frame(iframe)
sleep(2)
#main-section
page_url = driver.find_element(By.CSS_SELECTOR, 'a#hlNext').get_attribute('href')
print(page_url)

https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=2&ot=in


In [31]:
page_url_list = [] 
regex_page_url = "&page=[\d]*"
test = re.split(regex_page_url, page_url)
for each_page in range(1,26):
    page_url_list.append(f"{test[0]}&page={each_page}{test[1]}") 

In [32]:
# 放所有case的資料
all_case_list = []

# 開始爬case，這是每一頁的網址
for page in page_url_list:
    driver.get(page)
    case_url = driver.find_elements(By.CSS_SELECTOR, 'a#hlTitle')
    # 這是每一頁當中的至多20筆的資料
    for case in case_url:
        case.click()
        case_html = driver.page_source
        soup = bs(case_html, "lxml")

 
        # 案件標題id, 有四個標籤都是使用此選擇器，包含主文....(第四個)，那就抓這個就好了啊...
        all = soup.select("div.col-td")

        # 相關法條
        lines = soup.select_one('ul.rela-law')
        
        # 歷審裁判 history_judgement
        history_judgement = soup.select("div#JudHis div.panel-body ul li")
        history_judgement_list = []
        for index in range(0, len(history_judgement)):
            history_judgement_list.append({
                "case_id":history_judgement[index].get_text(),
                "url":"https://judgment.judicial.gov.tw/FJUD/"+history_judgement[index].select_one('a')['href']
            })

        all_case_list.append({
            "case_id":all[0].get_text().strip(),
            "date":all[1].get_text().strip(),
            "simple_reason":all[2].get_text().strip(),
            "context":all[3].get_text(),
            "laws":lines,
            "history_judgement":history_judgement_list
        })
        
        #此案件結束，回上一頁，點選下一個按件
        driver.back()

# 加try excecpt
# 開始爬case，這是每一頁的網址
for page in page_url_list:
    driver.get(page)
    try:
        cssSelector = 'a#hlTitle'
        WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, cssSelector)
        ))

        case_url = driver.find_elements(By.CSS_SELECTOR, cssSelector)
    except TimeoutException:
        print("cases not found")
        break

    # cases找到後開始一個一個點
    # 這是每一頁當中的至多20筆的資料    
    for case in case_url:
        case.click()
        #要邊要等網頁跑一下，不然javascript一樣出不來
        sleep(3)
        try:
            case_html = driver.page_source
            soup = bs(case_html, "lxml")

            allSelector = 'div.col-td'
            WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
            (By.CSS_SELECTOR, allSelector)
            ))
            all = soup.select(allSelector)


            linesSelector = 'ul.rela-law'
            WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
            (By.CSS_SELECTOR, linesSelector)
            ))
            lines = soup.select_one(linesSelector)
            history_judgement = soup.select("div#JudHis div.panel-body ul li")
            history_judgement_list = []
            for index in range(0, len(history_judgement)):
                history_judgement_list.append({
                    "case_id":history_judgement[index].get_text(),
                    "url":"https://judgment.judicial.gov.tw/FJUD/"+history_judgement[index].select_one('a')['href']
                })
            all_case_list.append({
            "case_id":all[0].get_text().strip(),
            "date":all[1].get_text().strip(),
            "simple_reason":all[2].get_text().strip(),
            "context":all[3].get_text(),
            "laws":lines,
            "history_judgement":history_judgement_list
            })

            driver.back()
            
        except TimeoutException:
            print("not found...element")
            continue


In [33]:
# 加try excecpt
# 要設一些如果沒抓到東西時的條件，比如歷史審判可能會沒有
# 然後改移下一開始的wait反正就是等旁偏那個javascript生出來的測攔出來就好
# 開始爬case，這是每一頁的網址
for page in page_url_list:
    driver.get(page)
    try:
        cssSelector = 'a#hlTitle'
        WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, cssSelector)
        ))

        case_url = driver.find_elements(By.CSS_SELECTOR, cssSelector)
    except TimeoutException:
        print("cases not found")
        break

    # cases找到後開始一個一個點
    # 這是每一頁當中的至多20筆的資料    
    for case in case_url:
        case.click()
        #要邊要等網頁跑一下，不然javascript一樣出不來
        try:
            # 看有沒有抓到js生成的側攔
            linesSelector = 'ul.rela-law'
            WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
            (By.CSS_SELECTOR, linesSelector)
            ))

            #有的話，就取得動態html
            case_html = driver.page_source
            soup = bs(case_html, "lxml")

        except TimeoutException:
            print("not found...element")
            continue

        allSelector = 'div.col-td'
        all = soup.select(allSelector)


        linesSelector = 'ul.rela-law'
        lines = soup.select_one(linesSelector)

        #目前出錯的都是這一塊
        history_judgement = soup.select("div#JudHis div.panel-body ul li")
        history_judgement_list = []
        try:
            for index in range(0, len(history_judgement)):
                history_judgement_list.append({
                    "case_id":history_judgement[index].get_text(),
                    "url":"https://judgment.judicial.gov.tw/FJUD/"+history_judgement[index].select_one('a')['href']
                })
        except TypeError: 
            # 如果沒有就pass就好，history_judgement就給她空list
            pass
        all_case_list.append({
        "case_id":all[0].get_text().strip(),
        "date":all[1].get_text().strip(),
        "simple_reason":all[2].get_text().strip(),
        "context":all[3].get_text(),
        "laws":lines,
        "history_judgement":history_judgement_list
        })

        driver.back()


In [36]:
print(all_case_list[44]['laws'])

<ul class="rela-law"></ul>


In [37]:
print(len(all_case_list))

500


In [39]:
import json

TypeError: write() argument must be str, not list